# Capstone Project - <i>"He Got Game So Show Him The Money"</i>

## Problem Statement

Player/agent negotiations with teams have the potential to be contentious which can cause anxiety to the player, team management as well as the fanbase. Time, energy and money are all at stake during contract negotiations so it would be of great benefit to have a means of accelerating this process by having a regression model that is agnostic to any biases and present a fair contract amount based on anticipated player production. Performance will be guided by r2 score. Baseline is the mean salary.

### Contents:
#### Part 1:
- [Methodology](#Methodology)
- [Data Acquisition](#Data-Acquisition)

### Methodology

I will use the NBA historical stats dataset from Kaggle and merge it with scraped salary information from hoopshype.com. I will begin with 10 years of data (2007-2017) which covers a decade of data that includes the last update of the NBA historical stats dataset. Once assembled, I will search for outliers for salary compared to winshares (WS) as well as minutes played. This should also remove outliers due to injury.

I will then scale the results as part of preprocessing and encode the position before checking for correlation and adding possible interaction terms. I will then set up train/test splits for modelling with an initial test size of 0.3.

I expect few incomplete records due to the quality of the source for the dataset (basketball-reference.com). I am cognizant that the author of the dataset may not have compiled everything without omissions. Additional challenges are that players that were traded during the season will have multiple rows which will need to be removed. I will use the 'TOT' row for that player instead which will contain the entire season's production. Some additional features that are available that haven't been added include player details (eg. which college they attended, height, weight, etc) will be considered for addition based on the initial model performance.

A major assumption is that we will only predict a single year's salary for our output. This may cause issues as contracts can run for multiple years which is beyond the scope of this project. Additionally, features such as "Bird Rights" and various salary cap exceptions will not be considered as these features are generally not widely tracked in NBA statistical databases.

The regressors I intend to use are Linear, Lasso, Ridge, Random Forest and SVR. Time permitting, I will also add in a neural network.

## Package Import

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import time


## Data Acquisition

### Test Webscrape With 2020 Salaries

In [2]:
url = "https://hoopshype.com/salaries/players/" #this is the only year that does not correspond with the site's convention of page naming for player salaries
req = requests.get(url)
req.status_code


200

In [3]:
html = req.text
soup = BeautifulSoup(html, 'lxml')

In [4]:
soup

<!DOCTYPE html>
<!--[if IE 8]> <html lang="en" class="no-js ie-browser lt-ie9"> <![endif]--><!--[if IE 9]> <html lang="en" class="no-js ie-browser ie9"> <![endif]--><!--[if gt IE 9]><!--><html class="no-js" lang="en"> <!--<![endif]-->
<head>
<script>(function(H){H.className=H.className.replace(/\bno-js\b/,'js')})(document.documentElement)</script>
<meta charset="utf-8"/>
<meta content="IE=Edge" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width, initial-scale=1.0" name="viewport"/>
<link href="http://gmpg.org/xfn/11" rel="profile"/>
<link href="https://hoopshype.com/xmlrpc.php" rel="pingback"/>
<!--[if lt IE 9]>
    <script src="https://s2.wp.com/wp-content/themes/vip/usatoday-lawrence/js/lib/html5.js" type="text/javascript"></script>
	<link rel="stylesheet" type="text/css" href="https://s2.wp.com/wp-content/themes/vip/usatoday-lawrence/ie8.css">
	<![endif]-->
<!--[if gte IE 9]>
	<link rel="stylesheet" type="text/css" href="https://s2.wp.com/wp-content/themes/vip/usatoday

In [5]:
print(soup.title)
print(soup.title.text)

<title>NBA Player Salaries | HoopsHype</title>
NBA Player Salaries | HoopsHype


In [6]:
salary2020 = []

all_td = soup.find_all('td')
for element in all_td:
    result = {}
    a_href = element.find('a')
    if a_href:
        result['name'] = a_href.text.strip()
        result['salary'] = element.find_next('td').attrs['data-value'] #this html tree navigation was key to acquiring the appropriate salary amount

    if len(result) == 2:
        salary2020.append(result)


In [7]:
salary2020

[{'name': 'Stephen Curry', 'salary': '40231758'},
 {'name': 'Russell Westbrook', 'salary': '38506482'},
 {'name': 'Chris Paul', 'salary': '38506482'},
 {'name': 'James Harden', 'salary': '38199000'},
 {'name': 'John Wall', 'salary': '38199000'},
 {'name': 'LeBron James', 'salary': '37436858'},
 {'name': 'Kevin Durant', 'salary': '37199000'},
 {'name': 'Blake Griffin', 'salary': '34449964'},
 {'name': 'Kyle Lowry', 'salary': '33296296'},
 {'name': 'Paul George', 'salary': '33005556'},
 {'name': 'Klay Thompson', 'salary': '32742000'},
 {'name': 'Kemba Walker', 'salary': '32742000'},
 {'name': 'Kawhi Leonard', 'salary': '32742000'},
 {'name': 'Jimmy Butler', 'salary': '32742000'},
 {'name': 'Tobias Harris', 'salary': '32742000'},
 {'name': 'Gordon Hayward', 'salary': '32700690'},
 {'name': 'Mike Conley', 'salary': '32511623'},
 {'name': 'Kyrie Irving', 'salary': '31742000'},
 {'name': 'Khris Middleton', 'salary': '30603448'},
 {'name': 'Paul Millsap', 'salary': '30350000'},
 {'name': 'Dam

### Function to Scrape Salaries

In [8]:
def salaryextract(year):
    url = "https://hoopshype.com/salaries/players/" + str(year-1) + "-" + str(year) + "/"
    req = requests.get(url)
    if req.status_code != 200:
        print (f'{url} failed. Status code {req.status_code}')
    html = req.text
    soup = BeautifulSoup(html, 'lxml')
    seasonsalary = []
    all_td = soup.find_all('td')
    for element in all_td:
        result = {}
        a_href = element.find('a')
        if a_href:
            result['name'] = a_href.text.strip()
            result['salary'] = element.find_next('td').attrs['data-value']

        if len(result) == 2:
            seasonsalary.append(result)
    return seasonsalary

In [9]:
# salary2019 = salaryextract(2019)
# salary2018 = salaryextract(2018)
salary2017 = salaryextract(2017)
time.sleep(5)
salary2016 = salaryextract(2016)
time.sleep(5)
salary2015 = salaryextract(2015)
time.sleep(10)
salary2014 = salaryextract(2014)
time.sleep(5)
salary2013 = salaryextract(2013)
time.sleep(5)
salary2012 = salaryextract(2012)
time.sleep(10)
salary2011 = salaryextract(2011)
time.sleep(5)
salary2010 = salaryextract(2010)
time.sleep(5)
salary2009 = salaryextract(2009)
time.sleep(10)
salary2008 = salaryextract(2008)

https://hoopshype.com/salaries/players/2010-2011/ failed. Status code 504


In [35]:
# hoopshype is timing out for this year's data. Error is on server side.
salary2011 = salaryextract(2011)

In [36]:
salary2011

[{'name': 'Kobe Bryant', 'salary': '24806250'},
 {'name': 'Rashard Lewis', 'salary': '19573711'},
 {'name': 'Kevin Garnett', 'salary': '18800000'},
 {'name': 'Tim Duncan', 'salary': '18700000'},
 {'name': 'Michael Redd', 'salary': '18300000'},
 {'name': 'Pau Gasol', 'salary': '17823000'},
 {'name': 'Andrei Kirilenko', 'salary': '17823000'},
 {'name': 'Gilbert Arenas', 'salary': '17730694'},
 {'name': 'Yao Ming', 'salary': '17686100'},
 {'name': 'Zach Randolph', 'salary': '17666666'},
 {'name': 'Vince Carter', 'salary': '17300000'},
 {'name': 'Dirk Nowitzki', 'salary': '17278618'},
 {'name': 'Carmelo Anthony', 'salary': '17149243'},
 {'name': 'Dwight Howard', 'salary': '16647180'},
 {'name': 'Kenyon Martin', 'salary': '16545454'},
 {'name': 'Amare Stoudemire', 'salary': '16486611'},
 {'name': 'Joe Johnson', 'salary': '16324500'},
 {'name': 'Elton Brand', 'salary': '15959099'},
 {'name': 'Predrag Stojakovic', 'salary': '15532320'},
 {'name': 'Chris Paul', 'salary': '14940153'},
 {'name':

### Import NBA Historical Statsitics Dataset

In [37]:
statsdf = pd.read_csv('./Data/Seasons_Stats.csv')

In [38]:
statsdf.columns #It is disappointing that per game stats were not included but these are easily calculated and can be added afterward

Index(['Unnamed: 0', 'Year', 'Player', 'Pos', 'Age', 'Tm', 'G', 'GS', 'MP',
       'PER', 'TS%', '3PAr', 'FTr', 'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%',
       'BLK%', 'TOV%', 'USG%', 'blanl', 'OWS', 'DWS', 'WS', 'WS/48', 'blank2',
       'OBPM', 'DBPM', 'BPM', 'VORP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%',
       '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB',
       'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS'],
      dtype='object')

In [39]:
statsdf = statsdf[statsdf['Year']>2007] #initially include only 10 years of data

In [40]:
statsdf.drop(columns = ['Unnamed: 0', 'blanl', 'blank2'], axis = 1, inplace = True)

In [41]:
statsdf.reset_index(drop = True, inplace = True)

In [42]:
statsdf.columns = statsdf.columns.str.lower()

In [43]:
statsdf.head()

,year,player,pos,age,tm,g,gs,mp,per,ts%,...,ft%,orb,drb,trb,ast,stl,blk,tov,pf,pts
0,2008.0,Shareef Abdur-Rahim,PF,31.0,SAC,6.0,0.0,51.0,5.5,0.317,...,1.000,6.0,4.0,10.0,4.0,1.0,0.0,1.0,9.0,10.0
1,2008.0,Arron Afflalo,SG,22.0,DET,75.0,9.0,970.0,10.2,0.494,...,0.782,36.0,101.0,137.0,52.0,31.0,8.0,34.0,83.0,276.0
2,2008.0,Maurice Ager,SG,23.0,TOT,26.0,3.0,165.0,1.8,0.363,...,0.500,4.0,9.0,13.0,8.0,0.0,1.0,6.0,17.0,51.0
3,2008.0,Maurice Ager,SG,23.0,DAL,12.0,3.0,77.0,-4.4,0.253,...,0.833,1.0,3.0,4.0,4.0,0.0,1.0,3.0,11.0,15.0
4,2008.0,Maurice Ager,SG,23.0,NJN,14.0,0.0,88.0,7.1,0.443,...,0.167,3.0,6.0,9.0,4.0,0.0,0.0,3.0,6.0,36.0


In [44]:
#Year and Age dtype conversion
statsdf['year'] = statsdf['year'].astype(int)
statsdf['age'] = statsdf['age'].astype(int)

### Set Salary into Dataframe

In [45]:
salary08df = pd.DataFrame(salary2008)
salary09df = pd.DataFrame(salary2009)
salary10df = pd.DataFrame(salary2010)
salary11df = pd.DataFrame(salary2011)
salary12df = pd.DataFrame(salary2012)
salary13df = pd.DataFrame(salary2013)
salary14df = pd.DataFrame(salary2014)
salary15df = pd.DataFrame(salary2015)
salary16df = pd.DataFrame(salary2016)
salary17df = pd.DataFrame(salary2017)

In [46]:
salary08df

,name,salary
0,Kevin Garnett,23750000
1,Michael Finley,21696750
2,Shaquille O'Neal,20000000
3,Jason Kidd,19728000
4,Jermaine O'Neal,19728000
...,...,...
464,Thomas Gardner,101451
465,Orien Greene,77061
466,Guillermo Diaz,56065
467,Stephane Lasme,48056


In [47]:
#function to remove html tags and convert contract amount to float. renamed name column to enable merge with statsdf
#tags and symbols subsequently removed at data acquisition so those sections are now redundant

def cleanup(df):
#     df['name'] = df['name'].str.replace('\\n','')
#     df['name'] = df['name'].str.replace('\\t','')
#     df['salary'] = df['salary'].str.replace('\\n','')    
#     df['salary'] = df['salary'].str.replace('\\t','')
#     df['salary'] = df['salary'].str.replace('$','')
#     df['salary'] = df['salary'].str.replace(',','').astype(float)
    df.rename(columns={"name": "player"}, inplace = True)
    return df


In [48]:
salary08df = cleanup(salary08df)
salary09df = cleanup(salary09df)
salary10df = cleanup(salary10df)
salary11df = cleanup(salary11df)
salary12df = cleanup(salary12df)
salary13df = cleanup(salary13df)
salary14df = cleanup(salary14df)
salary15df = cleanup(salary15df)
salary16df = cleanup(salary16df)
salary17df = cleanup(salary17df)


### Merge Stats DF with Salary DF

In [49]:
stats2008 = pd.merge(left = statsdf.loc[(statsdf['year'] == 2008)], right = salary08df, on = 'player')
stats2009 = pd.merge(left = statsdf.loc[(statsdf['year'] == 2009)], right = salary09df, on = 'player')
stats2010 = pd.merge(left = statsdf.loc[(statsdf['year'] == 2010)], right = salary10df, on = 'player')
stats2011 = pd.merge(left = statsdf.loc[(statsdf['year'] == 2011)], right = salary11df, on = 'player')
stats2012 = pd.merge(left = statsdf.loc[(statsdf['year'] == 2012)], right = salary12df, on = 'player')
stats2013 = pd.merge(left = statsdf.loc[(statsdf['year'] == 2013)], right = salary13df, on = 'player')
stats2014 = pd.merge(left = statsdf.loc[(statsdf['year'] == 2014)], right = salary14df, on = 'player')
stats2015 = pd.merge(left = statsdf.loc[(statsdf['year'] == 2015)], right = salary15df, on = 'player')
stats2016 = pd.merge(left = statsdf.loc[(statsdf['year'] == 2016)], right = salary16df, on = 'player')
stats2017 = pd.merge(left = statsdf.loc[(statsdf['year'] == 2017)], right = salary17df, on = 'player')

In [50]:
print (stats2008.shape)
print (stats2009.shape)
print (stats2010.shape)
print (stats2011.shape)
print (stats2012.shape)
print (stats2013.shape)
print (stats2014.shape)
print (stats2015.shape)
print (stats2016.shape)
print (stats2017.shape)

(562, 51)
(542, 51)
(540, 51)
(574, 51)
(494, 51)
(538, 51)
(561, 51)
(588, 51)
(534, 51)
(551, 51)


### Remove Duplicate Rows for Traded Players

In [51]:
#function to remove duplicate rows for traded players
def tradedplayerdrop(df):
    traded = [df['player'][i] for i in range(len(df['player'])) if df['tm'][i] == 'TOT']
    for i in df.index:
        if df['player'][i] in traded:
            if df['tm'][i] != 'TOT':
                df.loc[i,'tm'] = 'drop'

    df = df[df['tm'] != 'drop']
    return df

In [52]:
stats2008 = tradedplayerdrop(stats2008)
stats2009 = tradedplayerdrop(stats2009)
stats2010 = tradedplayerdrop(stats2010)
stats2011 = tradedplayerdrop(stats2011)
stats2012 = tradedplayerdrop(stats2012)
stats2013 = tradedplayerdrop(stats2013)
stats2014 = tradedplayerdrop(stats2014)
stats2015 = tradedplayerdrop(stats2015)
stats2016 = tradedplayerdrop(stats2016)
stats2017 = tradedplayerdrop(stats2017)

In [53]:
stats2015

,year,player,pos,age,tm,g,gs,mp,per,ts%,...,orb,drb,trb,ast,stl,blk,tov,pf,pts,salary
0,2015,Quincy Acy,PF,24,NYK,68.0,22.0,1287.0,11.9,0.533,...,79.0,222.0,301.0,68.0,27.0,22.0,60.0,147.0,398.0,915243
1,2015,Jordan Adams,SG,20,MEM,30.0,0.0,248.0,12.8,0.489,...,9.0,19.0,28.0,16.0,16.0,7.0,14.0,24.0,94.0,1344120
2,2015,Steven Adams,C,21,OKC,70.0,67.0,1771.0,14.1,0.549,...,199.0,324.0,523.0,66.0,38.0,86.0,99.0,222.0,537.0,2184960
3,2015,Jeff Adrien,PF,28,MIN,17.0,0.0,215.0,14.2,0.494,...,23.0,54.0,77.0,15.0,4.0,9.0,9.0,30.0,60.0,1217698
4,2015,Arron Afflalo,SG,29,TOT,78.0,72.0,2502.0,10.7,0.533,...,27.0,220.0,247.0,129.0,41.0,7.0,116.0,167.0,1035.0,7562500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
581,2015,James Young,SG,19,BOS,31.0,0.0,332.0,8.5,0.457,...,9.0,33.0,42.0,13.0,8.0,2.0,5.0,22.0,105.0,1674480
582,2015,Nick Young,SG,29,LAL,42.0,0.0,1000.0,14.2,0.520,...,17.0,79.0,96.0,41.0,23.0,11.0,44.0,83.0,563.0,4994420
583,2015,Thaddeus Young,PF,26,TOT,76.0,68.0,2434.0,15.7,0.507,...,127.0,284.0,411.0,173.0,124.0,25.0,117.0,171.0,1071.0,9160870
586,2015,Cody Zeller,C,22,CHO,62.0,45.0,1487.0,14.1,0.530,...,97.0,265.0,362.0,100.0,34.0,49.0,62.0,156.0,472.0,4030560


In [54]:
statsdf = pd.concat([stats2008, stats2009], axis = 0)
statsdf = pd.concat([statsdf, stats2010], axis = 0)
statsdf = pd.concat([statsdf, stats2011], axis = 0)
statsdf = pd.concat([statsdf, stats2012], axis = 0)
statsdf = pd.concat([statsdf, stats2013], axis = 0)
statsdf = pd.concat([statsdf, stats2014], axis = 0)
statsdf = pd.concat([statsdf, stats2015], axis = 0)
statsdf = pd.concat([statsdf, stats2016], axis = 0)
statsdf = pd.concat([statsdf, stats2017], axis = 0)


In [55]:
statsdf.reset_index(drop = True, inplace = True)

In [56]:
statsdf

,year,player,pos,age,tm,g,gs,mp,per,ts%,...,orb,drb,trb,ast,stl,blk,tov,pf,pts,salary
0,2008,Shareef Abdur-Rahim,PF,31,SAC,6.0,0.0,51.0,5.5,0.317,...,6.0,4.0,10.0,4.0,1.0,0.0,1.0,9.0,10.0,5800000
1,2008,Arron Afflalo,SG,22,DET,75.0,9.0,970.0,10.2,0.494,...,36.0,101.0,137.0,52.0,31.0,8.0,34.0,83.0,276.0,944520
2,2008,Maurice Ager,SG,23,TOT,26.0,3.0,165.0,1.8,0.363,...,4.0,9.0,13.0,8.0,0.0,1.0,6.0,17.0,51.0,974400
3,2008,LaMarcus Aldridge,PF,22,POR,76.0,76.0,2649.0,18.5,0.523,...,220.0,358.0,578.0,122.0,55.0,94.0,126.0,240.0,1350.0,4329360
4,2008,Malik Allen,PF,29,TOT,73.0,16.0,1096.0,9.8,0.500,...,66.0,130.0,196.0,44.0,19.0,31.0,45.0,149.0,338.0,998398
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4313,2017,Cody Zeller,PF,24,CHO,62.0,58.0,1725.0,16.7,0.604,...,135.0,270.0,405.0,99.0,62.0,58.0,65.0,189.0,639.0,5318313
4314,2017,Tyler Zeller,C,27,BOS,51.0,5.0,525.0,13.0,0.508,...,43.0,81.0,124.0,42.0,7.0,21.0,20.0,61.0,178.0,8000000
4315,2017,Stephen Zimmerman,C,20,ORL,19.0,0.0,108.0,7.3,0.346,...,11.0,24.0,35.0,4.0,2.0,5.0,3.0,17.0,23.0,950000
4316,2017,Paul Zipser,SF,22,CHI,44.0,18.0,843.0,6.9,0.503,...,15.0,110.0,125.0,36.0,15.0,16.0,40.0,78.0,240.0,750000


### Export Merged File

In [ ]:
#export the merged stat and salary df so we don't need to scrape again
statsdf.to_csv('./Data/stats_merged.csv', sep = ',', index=False)

### Additional Considerations

Considered adding player details such as height, weight, etc. but discovered that some players with the same name played at the same time. Need to verify if this affects dataset in use (2008-2017 seasons)

In [288]:
statsdf = pd.read_csv('./Data/stats_merged.csv')

In [289]:
players = pd.read_csv('./Data/Players.csv')
playerdata = pd.read_csv('./Data/player_data.csv')

In [290]:
players

,Unnamed: 0,Player,height,weight,collage,born,birth_city,birth_state
0,0,Curly Armstrong,180.0,77.0,Indiana University,1918.0,NaN,NaN
1,1,Cliff Barker,188.0,83.0,University of Kentucky,1921.0,Yorktown,Indiana
2,2,Leo Barnhorst,193.0,86.0,University of Notre Dame,1924.0,NaN,NaN
3,3,Ed Bartels,196.0,88.0,North Carolina State University,1925.0,NaN,NaN
4,4,Ralph Beard,178.0,79.0,University of Kentucky,1927.0,Hardinsburg,Kentucky
...,...,...,...,...,...,...,...,...
3917,3917,Troy Williams,198.0,97.0,South Carolina State University,1969.0,Columbia,South Carolina
3918,3918,Kyle Wiltjer,208.0,108.0,Gonzaga University,1992.0,Portland,Oregon
3919,3919,Stephen Zimmerman,213.0,108.0,"University of Nevada, Las Vegas",1996.0,Hendersonville,Tennessee
3920,3920,Paul Zipser,203.0,97.0,NaN,1994.0,Heidelberg,Germany


In [291]:
duplicatedrowsDF = players[players['Player'].duplicated()]

In [292]:
duplicatedrowsDF

,Unnamed: 0,Player,height,weight,collage,born,birth_city,birth_state


In [293]:
playerdata

,name,year_start,year_end,position,height,weight,birth_date,college
0,Alaa Abdelnaby,1991,1995,F-C,6-10,240.0,"June 24, 1968",Duke University
1,Zaid Abdul-Aziz,1969,1978,C-F,6-9,235.0,"April 7, 1946",Iowa State University
2,Kareem Abdul-Jabbar,1970,1989,C,7-2,225.0,"April 16, 1947","University of California, Los Angeles"
3,Mahmoud Abdul-Rauf,1991,2001,G,6-1,162.0,"March 9, 1969",Louisiana State University
4,Tariq Abdul-Wahad,1998,2003,F,6-6,223.0,"November 3, 1974",San Jose State University
...,...,...,...,...,...,...,...,...
4545,Ante Zizic,2018,2018,F-C,6-11,250.0,"January 4, 1997",NaN
4546,Jim Zoet,1983,1983,C,7-1,240.0,"December 20, 1953",Kent State University
4547,Bill Zopf,1971,1971,G,6-1,170.0,"June 7, 1948",Duquesne University
4548,Ivica Zubac,2017,2018,C,7-1,265.0,"March 18, 1997",NaN


In [294]:
duplicatedrowsDF = playerdata[playerdata['name'].duplicated()]

In [295]:
duplicatedrowsDF

,name,year_start,year_end,position,height,weight,birth_date,college
80,Dan Anderson,1975,1976,G,6-2,185.0,"January 1, 1951",University of Southern California
424,Bill Bradley,1968,1977,F-G,6-5,205.0,"July 28, 1943",Princeton University
488,Dee Brown,2007,2009,G,6-0,185.0,"August 17, 1984",University of Illinois at Urbana-Champaign
514,Roger Brown,1973,1980,C,6-11,225.0,"February 23, 1950",University of Kansas
925,Mark Davis,1996,2000,G-F,6-7,210.0,"April 26, 1973",Texas Tech University
929,Mike Davis,1983,1983,F,6-10,230.0,"August 2, 1956",University of Maryland
1055,Larry Drew,2015,2015,G,6-2,180.0,"March 5, 1990","University of California, Los Angeles"
1069,Bob Duffy,1963,1965,G,6-3,185.0,"September 26, 1940",Colgate University
1080,Mike Dunleavy,2003,2017,F-G,6-9,230.0,"September 15, 1980",Duke University
1191,Patrick Ewing,2011,2011,F,6-8,235.0,"May 20, 1984",Georgetown University


In [296]:
playerdata = playerdata[playerdata['year_end'] > 2005]
playerdata = playerdata[playerdata['year_start'] < 2018]

In [297]:
duplicatedrowsDF = playerdata[playerdata['name'].duplicated()]
duplicatedrowsDF

,name,year_start,year_end,position,height,weight,birth_date,college
2000,Chris Johnson,2013,2016,F-G,6-6,206.0,"April 29, 1990",University of Dayton
2787,Tony Mitchell,2014,2014,F,6-8,235.0,"April 7, 1992",University of North Texas
3146,Gary Payton,2017,2018,G,6-3,190.0,"December 1, 1992",Oregon State University
4393,Marcus Williams,2008,2009,F,6-7,205.0,"November 18, 1986",University of Arizona
4496,Chris Wright,2013,2013,G,6-1,210.0,"November 4, 1989",Georgetown University


In [298]:
playerdata[playerdata['name'] == 'Chris Johnson']

,name,year_start,year_end,position,height,weight,birth_date,college
1999,Chris Johnson,2011,2013,C,6-11,210.0,"July 15, 1985",Louisiana State University
2000,Chris Johnson,2013,2016,F-G,6-6,206.0,"April 29, 1990",University of Dayton


In [299]:
playerdata[playerdata['name'] == 'Tony Mitchell']

,name,year_start,year_end,position,height,weight,birth_date,college
2786,Tony Mitchell,2014,2014,F,6-6,216.0,"August 7, 1989",University of Alabama
2787,Tony Mitchell,2014,2014,F,6-8,235.0,"April 7, 1992",University of North Texas


In [300]:
playerdata[playerdata['name'] == 'Gary Payton']

,name,year_start,year_end,position,height,weight,birth_date,college
3145,Gary Payton,1991,2007,G,6-4,180.0,"July 23, 1968",Oregon State University
3146,Gary Payton,2017,2018,G,6-3,190.0,"December 1, 1992",Oregon State University


In [301]:
playerdata[playerdata['name'] == 'Marcus Williams']

,name,year_start,year_end,position,height,weight,birth_date,college
4392,Marcus Williams,2007,2010,G,6-3,205.0,"December 3, 1985",University of Connecticut
4393,Marcus Williams,2008,2009,F,6-7,205.0,"November 18, 1986",University of Arizona


In [302]:
playerdata[playerdata['name'] == 'Chris Wright']

,name,year_start,year_end,position,height,weight,birth_date,college
4495,Chris Wright,2012,2014,F,6-8,226.0,"September 30, 1988",University of Dayton
4496,Chris Wright,2013,2013,G,6-1,210.0,"November 4, 1989",Georgetown University


Reviewing both supplemental files shows that the 'players.csv' is less useful compared to the 'playerdata.csv' file. The 'playerdata' file contains nearly all of the same info but with improved detail.

In [303]:
playerdata.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1284 entries, 5 to 4548
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   name        1284 non-null   object 
 1   year_start  1284 non-null   int64  
 2   year_end    1284 non-null   int64  
 3   position    1284 non-null   object 
 4   height      1284 non-null   object 
 5   weight      1284 non-null   float64
 6   birth_date  1284 non-null   object 
 7   college     1077 non-null   object 
dtypes: float64(1), int64(2), object(5)
memory usage: 90.3+ KB


In [304]:
playerdata.reset_index(drop = True, inplace = True)

In [305]:
playerdata.loc[:,'height'] = [(int(playerdata['height'][i][0])*12 + int(playerdata['height'][i][2])) for i in range(len(playerdata))]

In [306]:
playerdata['college'].fillna('No College', inplace = True)

In [307]:
playerdata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1284 entries, 0 to 1283
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   name        1284 non-null   object 
 1   year_start  1284 non-null   int64  
 2   year_end    1284 non-null   int64  
 3   position    1284 non-null   object 
 4   height      1284 non-null   int64  
 5   weight      1284 non-null   float64
 6   birth_date  1284 non-null   object 
 7   college     1284 non-null   object 
dtypes: float64(1), int64(3), object(4)
memory usage: 80.4+ KB


In [308]:
statsdf.to_csv('./Data/playerdata_clean.csv', sep = ',', index=False)

In [309]:
playerdata.drop(columns = ['year_end', 'position', 'birth_date'], axis = 1, inplace = True)

In [310]:
playerdata

,name,year_start,height,weight,college
0,Shareef Abdur-Rahim,1997,81,225.0,University of California
1,Alex Abrines,2017,78,190.0,No College
2,Alex Acker,2006,77,185.0,Pepperdine University
3,Quincy Acy,2013,79,240.0,Baylor University
4,Hassan Adams,2007,76,220.0,University of Arizona
...,...,...,...,...,...
1279,Tyler Zeller,2013,84,253.0,University of North Carolina
1280,Derrick Zimmerman,2006,75,195.0,Mississippi State University
1281,Stephen Zimmerman,2017,84,240.0,"University of Nevada, Las Vegas"
1282,Paul Zipser,2017,80,215.0,No College


In [311]:
playerdata.rename(columns={"name": "player"}, inplace = True)

In [312]:
statsdf.shape

(4318, 51)

In [313]:
statsdf = pd.merge(left = statsdf, right = playerdata, on = 'player')

In [314]:
statsdf.shape

(4327, 55)

In [315]:
statsdf[statsdf['player'] == 'Chris Johnson']

,year,player,pos,age,tm,g,gs,mp,per,ts%,...,stl,blk,tov,pf,pts,salary,year_start,height,weight,college
3051,2011,Chris Johnson,C,25,TOT,14.0,1.0,138.0,10.2,0.554,...,3.0,9.0,10.0,21.0,33.0,164368,2011,73,210.0,Louisiana State University
3052,2011,Chris Johnson,C,25,TOT,14.0,1.0,138.0,10.2,0.554,...,3.0,9.0,10.0,21.0,33.0,164368,2013,78,206.0,University of Dayton
3053,2012,Chris Johnson,C,26,TOT,27.0,0.0,175.0,10.8,0.557,...,6.0,9.0,15.0,33.0,55.0,762195,2011,73,210.0,Louisiana State University
3054,2012,Chris Johnson,C,26,TOT,27.0,0.0,175.0,10.8,0.557,...,6.0,9.0,15.0,33.0,55.0,762195,2013,78,206.0,University of Dayton
3055,2013,Chris Johnson,C,27,MIN,30.0,0.0,284.0,18.4,0.650,...,7.0,28.0,13.0,49.0,117.0,508586,2011,73,210.0,Louisiana State University
3056,2013,Chris Johnson,C,27,MIN,30.0,0.0,284.0,18.4,0.650,...,7.0,28.0,13.0,49.0,117.0,508586,2013,78,206.0,University of Dayton
3057,2013,Chris Johnson,SF,22,MEM,8.0,0.0,102.0,10.7,0.560,...,4.0,0.0,3.0,5.0,29.0,508586,2011,73,210.0,Louisiana State University
3058,2013,Chris Johnson,SF,22,MEM,8.0,0.0,102.0,10.7,0.560,...,4.0,0.0,3.0,5.0,29.0,508586,2013,78,206.0,University of Dayton
3059,2014,Chris Johnson,SF,23,BOS,40.0,0.0,789.0,10.8,0.537,...,27.0,4.0,22.0,63.0,250.0,916099,2011,73,210.0,Louisiana State University
3060,2014,Chris Johnson,SF,23,BOS,40.0,0.0,789.0,10.8,0.537,...,27.0,4.0,22.0,63.0,250.0,916099,2013,78,206.0,University of Dayton


In [316]:
#Identify appropriate player by referring to position vs playerdata
statsdf.drop(index= [3052, 3054, 3056, 3057, 3059], axis = 0, inplace = True)

In [317]:
statsdf[statsdf['player'] == 'Chris Johnson']

,year,player,pos,age,tm,g,gs,mp,per,ts%,...,stl,blk,tov,pf,pts,salary,year_start,height,weight,college
3051,2011,Chris Johnson,C,25,TOT,14.0,1.0,138.0,10.2,0.554,...,3.0,9.0,10.0,21.0,33.0,164368,2011,73,210.0,Louisiana State University
3053,2012,Chris Johnson,C,26,TOT,27.0,0.0,175.0,10.8,0.557,...,6.0,9.0,15.0,33.0,55.0,762195,2011,73,210.0,Louisiana State University
3055,2013,Chris Johnson,C,27,MIN,30.0,0.0,284.0,18.4,0.650,...,7.0,28.0,13.0,49.0,117.0,508586,2011,73,210.0,Louisiana State University
3058,2013,Chris Johnson,SF,22,MEM,8.0,0.0,102.0,10.7,0.560,...,4.0,0.0,3.0,5.0,29.0,508586,2013,78,206.0,University of Dayton
3060,2014,Chris Johnson,SF,23,BOS,40.0,0.0,789.0,10.8,0.537,...,27.0,4.0,22.0,63.0,250.0,916099,2013,78,206.0,University of Dayton


In [318]:
statsdf[statsdf['player'] == 'Tony Mitchell']

,year,player,pos,age,tm,g,gs,mp,per,ts%,...,stl,blk,tov,pf,pts,salary,year_start,height,weight,college
3882,2014,Tony Mitchell,SF,24,MIL,3.0,0.0,10.0,30.9,0.60,...,1.0,0.0,0.0,0.0,6.0,490180,2014,78,216.0,University of Alabama
3883,2014,Tony Mitchell,SF,24,MIL,3.0,0.0,10.0,30.9,0.60,...,1.0,0.0,0.0,0.0,6.0,490180,2014,80,235.0,University of North Texas
3884,2014,Tony Mitchell,SF,24,MIL,3.0,0.0,10.0,30.9,0.60,...,1.0,0.0,0.0,0.0,6.0,28834,2014,78,216.0,University of Alabama
3885,2014,Tony Mitchell,SF,24,MIL,3.0,0.0,10.0,30.9,0.60,...,1.0,0.0,0.0,0.0,6.0,28834,2014,80,235.0,University of North Texas
3886,2014,Tony Mitchell,PF,21,DET,21.0,0.0,79.0,18.1,0.54,...,6.0,3.0,4.0,9.0,22.0,490180,2014,78,216.0,University of Alabama
3887,2014,Tony Mitchell,PF,21,DET,21.0,0.0,79.0,18.1,0.54,...,6.0,3.0,4.0,9.0,22.0,490180,2014,80,235.0,University of North Texas
3888,2014,Tony Mitchell,PF,21,DET,21.0,0.0,79.0,18.1,0.54,...,6.0,3.0,4.0,9.0,22.0,28834,2014,78,216.0,University of Alabama
3889,2014,Tony Mitchell,PF,21,DET,21.0,0.0,79.0,18.1,0.54,...,6.0,3.0,4.0,9.0,22.0,28834,2014,80,235.0,University of North Texas


In [319]:
#Appropriate rows to drop identified by reviewing each player's wikipedia page
statsdf.drop(index= [3882, 3883, 3885, 3886, 3888, 3889], axis = 0, inplace = True)

In [320]:
statsdf[statsdf['player'] == 'Tony Mitchell']

,year,player,pos,age,tm,g,gs,mp,per,ts%,...,stl,blk,tov,pf,pts,salary,year_start,height,weight,college
3884,2014,Tony Mitchell,SF,24,MIL,3.0,0.0,10.0,30.9,0.60,...,1.0,0.0,0.0,0.0,6.0,28834,2014,78,216.0,University of Alabama
3887,2014,Tony Mitchell,PF,21,DET,21.0,0.0,79.0,18.1,0.54,...,6.0,3.0,4.0,9.0,22.0,490180,2014,80,235.0,University of North Texas


In [322]:
statsdf[statsdf['player'] == 'Gary Payton']

,year,player,pos,age,tm,g,gs,mp,per,ts%,...,stl,blk,tov,pf,pts,salary,year_start,height,weight,college


In [323]:
statsdf[statsdf['player'] == 'Marcus Williams']

,year,player,pos,age,tm,g,gs,mp,per,ts%,...,stl,blk,tov,pf,pts,salary,year_start,height,weight,college
2150,2008,Marcus Williams,PF-SF,21,TOT,11.0,0.0,36.0,6.8,0.250,...,1.0,1.0,0.0,3.0,10.0,1193090,2007,75,205.0,University of Connecticut
2151,2008,Marcus Williams,PF-SF,21,TOT,11.0,0.0,36.0,6.8,0.250,...,1.0,1.0,0.0,3.0,10.0,1193090,2008,79,205.0,University of Arizona
2152,2009,Marcus Williams,PG,23,GSW,9.0,0.0,54.0,3.7,0.305,...,1.0,1.0,4.0,6.0,12.0,1262520,2007,75,205.0,University of Connecticut
2153,2009,Marcus Williams,PG,23,GSW,9.0,0.0,54.0,3.7,0.305,...,1.0,1.0,4.0,6.0,12.0,1262520,2008,79,205.0,University of Arizona
2154,2009,Marcus Williams,PF,22,SAS,2.0,0.0,3.0,53.0,1.000,...,0.0,0.0,0.0,1.0,4.0,1262520,2007,75,205.0,University of Connecticut
2155,2009,Marcus Williams,PF,22,SAS,2.0,0.0,3.0,53.0,1.000,...,0.0,0.0,0.0,1.0,4.0,1262520,2008,79,205.0,University of Arizona
2156,2010,Marcus Williams,PG,24,MEM,62.0,1.0,872.0,10.6,0.456,...,32.0,1.0,74.0,43.0,269.0,855189,2007,75,205.0,University of Connecticut
2157,2010,Marcus Williams,PG,24,MEM,62.0,1.0,872.0,10.6,0.456,...,32.0,1.0,74.0,43.0,269.0,855189,2008,79,205.0,University of Arizona


In [324]:
statsdf.drop(index= [2150, 2153, 2154, 2157], axis = 0, inplace = True)

In [325]:
statsdf[statsdf['player'] == 'Marcus Williams']

,year,player,pos,age,tm,g,gs,mp,per,ts%,...,stl,blk,tov,pf,pts,salary,year_start,height,weight,college
2151,2008,Marcus Williams,PF-SF,21,TOT,11.0,0.0,36.0,6.8,0.250,...,1.0,1.0,0.0,3.0,10.0,1193090,2008,79,205.0,University of Arizona
2152,2009,Marcus Williams,PG,23,GSW,9.0,0.0,54.0,3.7,0.305,...,1.0,1.0,4.0,6.0,12.0,1262520,2007,75,205.0,University of Connecticut
2155,2009,Marcus Williams,PF,22,SAS,2.0,0.0,3.0,53.0,1.000,...,0.0,0.0,0.0,1.0,4.0,1262520,2008,79,205.0,University of Arizona
2156,2010,Marcus Williams,PG,24,MEM,62.0,1.0,872.0,10.6,0.456,...,32.0,1.0,74.0,43.0,269.0,855189,2007,75,205.0,University of Connecticut


In [326]:
statsdf[statsdf['player'] == 'Chris Wright']

,year,player,pos,age,tm,g,gs,mp,per,ts%,...,stl,blk,tov,pf,pts,salary,year_start,height,weight,college
3475,2012,Chris Wright,SF,23,GSW,24.0,1.0,186.0,18.6,0.597,...,7.0,13.0,8.0,22.0,70.0,473604,2012,80,226.0,University of Dayton
3476,2012,Chris Wright,SF,23,GSW,24.0,1.0,186.0,18.6,0.597,...,7.0,13.0,8.0,22.0,70.0,473604,2013,73,210.0,Georgetown University
3477,2013,Chris Wright,SG,23,DAL,3.0,0.0,4.0,-1.9,0.500,...,0.0,0.0,1.0,0.0,2.0,27859,2012,80,226.0,University of Dayton
3478,2013,Chris Wright,SG,23,DAL,3.0,0.0,4.0,-1.9,0.500,...,0.0,0.0,1.0,0.0,2.0,27859,2013,73,210.0,Georgetown University
3479,2014,Chris Wright,SF,25,MIL,8.0,0.0,126.0,16.5,0.577,...,7.0,5.0,5.0,17.0,48.0,139212,2012,80,226.0,University of Dayton
3480,2014,Chris Wright,SF,25,MIL,8.0,0.0,126.0,16.5,0.577,...,7.0,5.0,5.0,17.0,48.0,139212,2013,73,210.0,Georgetown University


In [327]:
statsdf.drop(index= [3476, 3477, 3480], axis = 0, inplace = True)

In [328]:
statsdf[statsdf['player'] == 'Chris Wright']

,year,player,pos,age,tm,g,gs,mp,per,ts%,...,stl,blk,tov,pf,pts,salary,year_start,height,weight,college
3475,2012,Chris Wright,SF,23,GSW,24.0,1.0,186.0,18.6,0.597,...,7.0,13.0,8.0,22.0,70.0,473604,2012,80,226.0,University of Dayton
3478,2013,Chris Wright,SG,23,DAL,3.0,0.0,4.0,-1.9,0.500,...,0.0,0.0,1.0,0.0,2.0,27859,2013,73,210.0,Georgetown University
3479,2014,Chris Wright,SF,25,MIL,8.0,0.0,126.0,16.5,0.577,...,7.0,5.0,5.0,17.0,48.0,139212,2012,80,226.0,University of Dayton


In [329]:
statsdf.reset_index(drop = True, inplace = True)

In [330]:
#export the merged stats with additional player details
statsdf.to_csv('./Data/stats_enriched.csv', sep = ',', index=False)